# DST Airlines

[LINK](https://docs.google.com/document/d/1wyo5QOe8UZZE0_fm0PNdc_bTUNqjyrBMHSHwH-JfSEM/edit)

Description détaillée :  
De nos jours, il est possible d’avoir des informations sur les vols dans le monde entier et de traquer en temps réel un avion. Nous pouvons observer ce site en guise d’exemple.  
Le but ici est de s’en approcher le plus possible en passant par des API de différentes compagnies aériennes.

Etape | Description | Modules / Masterclass / Templates | Objectif | Conditions de validation du projet
----- | ----------- | --------------------------------- | -------- | ---------------------------------
1 | Exploration de données non structurées | Passer par l’API de Lufthansa pour récupérer des données sur les vols | 103 - Introduction aux APIs de données | Un fichier explicatif du traitement et des différentes données accessibles (doc / pdf). Un exemple de données collectées.


[Lufthansa Open API Documentation](https://developer.lufthansa.com/docs)

[LH OpenAPI testing](https://developer.lufthansa.com/io-docs)

[Public API Method Details](https://developer.lufthansa.com/docs/read/api_details)

!!! [Un repo GIT englobe déjà les requêtes à l'API, il suffirait de le forker](https://github.com/arossmann/lh_open_api)

In [ ]:
from Lhapi import Lhapi as api

test = api()


In [ ]:
countries = test.request_api('v1', '/mds-references/countries')

In [ ]:
countries

In [ ]:
import re, json

pattern = r'{"@Href":.*,"@Rel":"next"}'
res = '[' + re.search(pattern, countries).group() +']'
res = json.loads(res)
link = None
for meta in res:
    if meta["@Rel"] == "next":
        link = meta['@Href']
print(link)



In [ ]:
cities = test.request_api('v1', '/mds-references/cities')

In [ ]:
print(cities)

In [ ]:
airports = test.request_api('v1', '/mds-references/airports')

In [ ]:
print(airports)

In [ ]:
airlines = test.request_api('v1', '/mds-references/airlines?limit=2')

In [ ]:
print(airlines)

In [ ]:
aircrafts = test.request_api('v1', '/mds-references/aircraft?limit=3')

In [ ]:
print(aircrafts)

In [ ]:
test.request_file('aircrafts.json', 'v1', '/mds-references/aircraft?limit=100', 'AircraftSummary', full=True)

In [ ]:
# REFERENCES
# https://developer.lufthansa.com/docs/read/api_details/Reference_Data
# missing nearest airport

reference_items = ['countries', 'cities', 'airports', 'airlines', 'aircraft']
for ref in reference_items:
    test.request_file(ref + '.json', 'v1', '/mds-references/' + ref)

In [ ]:
# get airports where LH is present
test.request_file('airports_lh.json', 'v1', '/mds-references/airports?LHoperated=1')

In [ ]:
# get lounges from airports where LH is present
import json

def read_file(file_path, word):
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip().startswith('"' + word + '"'):
                res = line.split(':')[1].replace(',', '').replace('"', '').strip()
                yield res


airport_lh_codes = read_file("files/airports_lh.json", "AirportCode")
for code in airport_lh_codes:
    print(test.request_api("/offers/lounges/" + code))


In [ ]:
# OFFERS



In [ ]:
# test api v2
req = test.request_api('v2', '/customer-deeplinks/servicing-links/booking')
print(req)

In [ ]:
print(test.get_partner_token())

In [ ]:
# Test vietnam schedules

req = test.request_api('v1', '/operations/schedules/FRA/SGN/2023-11-29?directFlights=true')
print(req)